In [95]:
#-*- coding : utf-8 -*-
# coding: utf-8
import seaborn as sns
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
from statsmodels.api import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
df = pd.DataFrame(pd.read_csv('binary.csv', encoding='gbk'))#%%



In [96]:
df.head()
df.columns = ["admit", "gre", "gpa", "prestige"]
df

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [97]:
#虚拟变量分4类,为了避免多重共线性只留3个
data=df.drop('prestige',axis=1).join(data.iloc[:,:3],how='outer')
data['intercept'] = 1.0
data

,admit,gre,gpa,prestige_1,prestige_2,prestige_3,intercept
0,0,380,3.61,0,0,1,1.0
1,1,660,3.67,0,0,1,1.0
2,1,800,4.00,1,0,0,1.0
3,1,640,3.19,0,0,0,1.0
4,0,520,2.93,0,0,0,1.0
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,1.0
396,0,560,3.04,0,0,1,1.0
397,0,460,2.63,0,1,0,1.0
398,0,700,3.65,0,1,0,1.0


In [98]:
# 指定作为训练变量的列，不含目标列`admit`
train_cols = data.columns[1:]
# Index([gre, gpa, prestige_2, prestige_3, prestige_4], dtype=object)

logit = sm.Logit(data['admit'], data[train_cols])
# 拟合模型
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6


In [99]:
import copy
combos = copy.deepcopy(data)

# 数据中的列要跟预测时用到的列一致
predict_cols = combos.columns[1:]

# 预测集也要添加intercept变量
combos['intercept'] = 1.0

# 进行预测，并将预测评分存入 predict 列中
combos['predict'] = result.predict(combos[predict_cols])
a=combos['predict']
b=combos['admit']

In [104]:
total = 0
for i in range(0,len(a)):
    if b[i]==round(a[i]):
        total+=1
percent=100*total/len(a)
print('准确率为{0}%'.format(percent))
result.summary()


准确率为71.0%


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Fri, 23 Jul 2021   Pseudo R-squ.:                 0.08292
Time:                        23:25:14   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
Covariance Type:            nonrobust   LLR p-value:                 7.578e-08
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0023      0.001      2.070      0.038       0.000       0.004
gpa            0.8040      0.332      2.423      0.015       0.154       1.454
prestige_1     1.5515      0.418      3.713      0.000       0.733       2.370
prestige_2     0.8760      0.367      2.389      0.017       0.157       1.595
prestige_3     0.2113      0.393      0.538      0.591      -0.559       0.981
intercept     -5.5414      1.138     -4.869      0.000      -7.772      -3.311
==============================================================================
"""